In [1]:
import os

import open3d
from plyfile import PlyData, PlyElement
import numpy as np
import pandas as pd

import torch
torch.cuda.is_available()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


True

In [2]:
train_files = []
with open('./dataset/train.txt') as f:
    for line in f:
        train_files.append(line.strip())

In [3]:
def read_plyfile(filepath):
    """
    Read ply file and return it as np.array. Returns None if empty
    """
    with open(filepath, 'rb') as f:
        plydata = PlyData.read(f)
    if plydata.elements:
        return pd.DataFrame(plydata.elements[0].data).values

In [4]:
train_root = './dataset'
path = os.path.join(train_root, train_files[0])

data = read_plyfile(path)

In [5]:
data[:, :3]

array([[-3.41029572,  1.22604847,  0.19869943],
       [-3.39742899,  1.25844002,  0.19795202],
       [-3.41037989,  1.1878475 ,  0.10961316],
       ...,
       [ 3.32770324, -0.70443457,  2.51359797],
       [ 3.33274317, -0.67037743,  2.5465734 ],
       [ 3.3289566 , -0.70229721,  2.56173825]])

In [50]:
data[:, 3:6]

array([[101., 107.,  90.],
       [ 88.,  83.,  78.],
       [ 39.,  39.,  35.],
       ...,
       [242., 212., 144.],
       [175., 161., 127.],
       [179., 164., 128.]])

In [35]:
data.shape
# (x, y, z, r, g, b, label, instance)

(81369, 8)

In [36]:
data[:, :3].shape
# (x, y, z)

(81369, 3)

In [40]:
data[:, 3:6]

array([[101., 107.,  90.],
       [ 88.,  83.,  78.],
       [ 39.,  39.,  35.],
       ...,
       [242., 212., 144.],
       [175., 161., 127.],
       [179., 164., 128.]])

In [18]:
len(data)

81369

# Read normal

In [8]:
def load_ply(filepath):
    with open(filepath, "rb") as f:
        plydata = PlyData.read(f)
    data = plydata.elements[0].data
    coords = np.array([data["x"], data["y"], data["z"]], dtype=np.float32).T
    feats = None
    labels = None
    if ({"red", "green", "blue"} - set(data.dtype.names)) == set():
        feats = np.array([data["red"], data["green"], data["blue"]], dtype=np.uint8).T
    if "label" in data.dtype.names:
        labels = np.array(data["label"], dtype=np.uint32)
    return coords, feats, labels

def load_ply_with_normals(filepath):
    mesh = open3d.io.read_triangle_mesh(str(filepath))
    if not mesh.has_vertex_normals():
        mesh.compute_vertex_normals()
    vertices = np.asarray(mesh.vertices)
    normals = np.asarray(mesh.vertex_normals)
    print(vertices)
    print("="*10)
    print(normals)

    coords, feats, labels = load_ply(filepath)
    assert np.allclose(coords, vertices), "different coordinates"
    feats = np.hstack((feats, normals))

    return coords, feats, labels

In [9]:
train_root = './dataset'
path = os.path.join(train_root, train_files[0])

data = load_ply_with_normals(path)

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
[[-3.41029572  1.22604847  0.19869943]
 [-3.39742899  1.25844002  0.19795202]
 [-3.41037989  1.1878475   0.10961316]
 ...
 [ 3.32770324 -0.70443457  2.51359797]
 [ 3.33274317 -0.67037743  2.5465734 ]
 [ 3.3289566  -0.70229721  2.56173825]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [12]:
data[2]

array([21, 21, 21, ...,  1,  1,  1], dtype=uint32)

# ARB-loss
- Ref: https://gist.github.com/yang-zhang/217dcc6ae9171d7a46ce42e215c1fee0

In [3]:
import torch
import torch.nn.functional as F

In [9]:
batch_size, n_classes = 5, 3
x = torch.randn(batch_size, n_classes)
# (bs, C)
x.shape, x

(torch.Size([5, 3]),
 tensor([[ 0.3511, -0.3870, -0.1297],
         [ 1.6277,  1.5000, -0.1872],
         [ 0.3714, -0.2828,  1.0229],
         [ 1.6189,  1.1331, -1.2980],
         [ 0.4319,  0.8472,  0.6264]]))

In [15]:
target = torch.randint(n_classes, size=(batch_size,), dtype=torch.long)
target

tensor([2, 2, 0, 1, 0])

In [16]:
weight_class = torch.bincount(target) / target.size(0)
weight_class

tensor([0.4000, 0.2000, 0.4000])

In [17]:
pred = F.log_softmax(x, dim=-1)
loss = F.nll_loss(pred, target)
loss

tensor(1.4600)

In [18]:
pred = F.log_softmax(x, dim=-1)
loss = F.nll_loss(pred, target, weight=weight_class)
loss

tensor(1.5113)

# Compute color mean and color std

In [23]:
def read_txt(path):
    data = []
    with open(path) as f:
        contents = f.readlines()
        for content in contents:
            data.append(content.strip())
    return data

In [28]:
root = './dataset'
train_path, val_path = './train_split.txt', './val_split.txt'
train_data_path, val_data_path = read_txt(train_path), read_txt(val_path)

In [48]:
train_data, val_data = [], []
for dp in train_data_path:
    dp = os.path.join(root, dp)
    train_data.append(
        # get only RGB
        read_plyfile(dp)[:, 3:6]
        )
for dp in val_data_path:
    dp = os.path.join(root, dp)
    val_data.append(
        # get only RGB
        read_plyfile(dp)[:, 3:6]
        )

In [60]:
train_color_mean, val_color_mean = [], []
train_color_std, val_color_std = [], []

for rgb in train_data:
    train_color_mean.append(
        [
            float((rgb[:, 0] / 255).mean()),
            float((rgb[:, 1] / 255).mean()),
            float((rgb[:, 2] / 255).mean()),
        ]
    )
    train_color_std.append(
        [
            float(((rgb[:, 0] / 255) ** 2).mean()),
            float(((rgb[:, 1] / 255) ** 2).mean()),
            float(((rgb[:, 2] / 255) ** 2).mean()),
        ]
    )

for rgb in val_data:
    val_color_mean.append(
        [
            float((rgb[:, 0] / 255).mean()),
            float((rgb[:, 1] / 255).mean()),
            float((rgb[:, 2] / 255).mean()),
        ]
    )
    val_color_std.append(
        [
            float(((rgb[:, 0] / 255) ** 2).mean()),
            float(((rgb[:, 1] / 255) ** 2).mean()),
            float(((rgb[:, 2] / 255) ** 2).mean()),
        ]
    )

In [61]:
def compute_color_mean_std(color_mean, color_std):
    color_mean = np.array(color_mean).mean(axis=0)
    color_std = np.sqrt(np.array(color_std).mean(axis=0) - color_mean ** 2)
    feats_mean_std = {
        "mean": [float(each) for each in color_mean],
        "std": [float(each) for each in color_std],
    }
    return feats_mean_std

In [66]:
train_color_mean_std = compute_color_mean_std(
    train_color_mean, train_color_std
)
val_color_mean_std = compute_color_mean_std(
    val_color_mean, val_color_std
)

({'mean': [0.4764729909316615, 0.42902326580153166, 0.37332420105708963],
  'std': [0.2837346299860227, 0.2754832225183542, 0.2691509161776033]},
 {'mean': [0.45397303778149745, 0.4050079317532427, 0.35046137096670843],
  'std': [0.27850010265525965, 0.2683205838692733, 0.26240199032866635]})

In [70]:
with open('train_color_mean_std.npy', 'wb') as f:
    np.save(f, train_color_mean_std['mean'])
    np.save(f, train_color_mean_std['std'])

with open('val_color_mean_std.npy', 'wb') as f:
    np.save(f, val_color_mean_std['mean'])
    np.save(f, val_color_mean_std['std'])

In [71]:
with open('train_color_mean_std.npy', 'rb') as f:
    train_color_mean = np.load(f)
    train_color_std = np.load(f)

with open('val_color_mean_std.npy', 'rb') as f:
    val_color_mean = np.load(f)
    val_color_std = np.load(f)

In [72]:
train_color_mean, train_color_std, \
    val_color_mean, val_color_std

(array([0.47647299, 0.42902327, 0.3733242 ]),
 array([0.28373463, 0.27548322, 0.26915092]),
 array([0.45397304, 0.40500793, 0.35046137]),
 array([0.2785001 , 0.26832058, 0.26240199]))

# Test data aug

In [3]:
from a_3l2h.data_aug import SemanticSegmentationDataset

ImportError: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/guest/r11922a05/anaconda3/envs/lg_semseg/lib/python3.8/site-packages/MinkowskiEngineBackend/_C.cpython-38-x86_64-linux-gnu.so)